In [ ]:
# Setting up a custom stylesheet in IJulia
file = open("style.css") # A .css file in the same folder as this notebook file
style = read(file, String) # Read the file
HTML("$style") # Output as HTML

In [1]:
# Checking the kernel
# These are uncertain times :)
+(2, 2)

4

# Using Julia 1.0 for introductory statistics

## Introduction

Julia is well suited to general purpose data analysis.  It has many built-in statistical functions and there are many packages that greatly extend the capabilities of Julia as a scientific programming language for statistics.

In this section we will take a look at introductory statistics using Julia 1.0.  There are differences between this version of Julia and version 0.6.  Not all the packages that are available for version 0.6 and prior, are quite ready for version 1.0.  At the time of recording the packages that are used in this section do all compile and can be used.

In the first part of this section of the course, we will take a look at creating our own data for statistical analysis.  It is great to be able to generate simulated data, especially when you are just starting out and might not have access to proper datasets.

When viewing a new dataset, it is alway good to start by describing it.  Human beings are not designed to look at large tables of data and understand what it is trying to tell us.  Using summarizing, or descriptive, statistics helps us to gain an insight into the data before we start to analyze it.

This section will also look at visualizing data.  It many cases, this allows for an even better understanding of the data.

The `HypothesisTests` and `GLM` packages allow us to do many common statistical tests and we will have a look at Student's _t_ test, linear regression models, and the $\chi^2$ test for independence.

We will conclude with a look at exporting our data in the form of a spreadsheet.  Let's start, though, by importing the packages that we will be using.

## Adding packages

If the packages that are listed below are not installed on your system, then do the following for each package, i.e. `PyPlot`.

```
using Pkg;
Pkg.add("PyPlot")
```

In [1]:
import Pkg

Pkg.add("Distributions")
Pkg.add("StatsBase")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("HypothesisTests")
Pkg.add("StatsPlots")
Pkg.add("GLM")
Pkg.add("GR")

   Resolving package versions...
  No Changes to `C:\Users\chadi\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\chadi\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\chadi\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\chadi\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\chadi\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\chadi\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\chadi\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\chadi\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
   Installed HypothesisTests ─ v0.11.0
    Updating `C:\Users\chadi\.julia\environments\v1.10\Project.toml`
  [09f84164] + HypothesisTests v0.11.0
    Updating `C:\Users\chadi\.julia\environments\v1.10\Manifest.toml`
  [861a8166] + Combinatorics v1.0.2
  

In [2]:
using Distributions    # Create random variables
using StatsBase        # Basic statistical support
using CSV              # Reading and writing CSV files
using DataFrames       # Create a data structure
using HypothesisTests  # Perform statistical tests
using StatsPlots       # Statistical plotting
using GLM              # General linear models

We recommend using GR backend instead of PyPlot backend.

In [4]:
gr()                   # Use GR Backend

UndefVarError: UndefVarError: `gr` not defined

## Creating random variable

We mentioned in the introduction that the creation of simulated data is a great way to start learning how to use Julia for statistics.  In the code below, we create five variables with random data point values.

In [5]:
age = rand(18:80, 100);  # Uniform distribution
wcc = round.(rand(Distributions.Normal(12, 2), 100), digits = 1)  # Normal distribution & round to one decimal place
crp = round.(Int, rand(Distributions.Chisq(4), 100)) .* 10  # Chi-squared distribution with broadcasting & alternate round()
treatment = rand(["A", "B"], 100); # Uniformly weighted
result = rand(["Improved", "Static", "Worse"], 100);  # Uniformly weighted

## Descriptive statistics

While there are common statistical function in Julia such as `mean()` and `std()`, it is more convenient to use the `describe()` function from the `StatsBase` package.

In [6]:
# Mean of the age variable
mean(age)

51.74

In [7]:
# Median of age variable
median(age)

54.5

In [8]:
# Standard deviation of age
std(age)

18.248354014607024

In [9]:
# Variance of age
var(age)

333.00242424242424

In [10]:
# Mean of wcc
mean(wcc)

11.634

In [11]:
# Standard deviation of wcc
std(wcc)

2.023359542539051

In [12]:
# Descriptive statistics of the age variable
StatsBase.describe(age)

Summary Stats:
Length:         100
Missing Count:  0
Mean:           51.740000
Std. Deviation: 18.248354
Minimum:        18.000000
1st Quartile:   35.750000
Median:         54.500000
3rd Quartile:   68.000000
Maximum:        80.000000
Type:           Int64


In [13]:
# The summarystats() function omits the length and type
StatsBase.summarystats(wcc)

Summary Stats:
Length:         100
Missing Count:  0
Mean:           11.634000
Std. Deviation: 2.023360
Minimum:        7.300000
1st Quartile:   10.400000
Median:         12.000000
3rd Quartile:   13.125000
Maximum:        16.100000


## Creating a dataframe

When creating simulated data, it is best to store it in a dataframe object for easier manipulation.

In [14]:
data = DataFrame(Age = age, WCC = wcc, CRP = crp, Treatment = treatment, Result = result);

In [15]:
# Number of rows and columns
size(data)

(100, 5)

In [16]:
# First six rows. Note that head() method has been deprecated.
first(data, 6)

Row,Age,WCC,CRP,Treatment,Result
,Int64,Float64,Int64,String,String
1,72,11.2,80,B,Improved
2,35,14.7,20,A,Static
3,68,13.2,100,B,Worse
4,58,11.2,30,A,Worse
5,76,12.7,30,B,Static
6,59,11.2,20,B,Improved


We can create dataframe objects by selecting only subjects according to their data point values for a particular variable. 

**Note that dataframe slicing using `data[data[:Treatment] .== "A", :]` has been deprecated from the video lecture. Use below slicing method.**

In [17]:
dataA = data[data[:, :Treatment] .== "A", :]   # Only patient in treatment group A
dataB = data[data[:, :Treatment] .== "B", :];  # Only patient in treatment group B

In [18]:
# Show first 5 rows from dataA
first(dataA, 5)

Row,Age,WCC,CRP,Treatment,Result
,Int64,Float64,Int64,String,String
1,35,14.7,20,A,Static
2,58,11.2,30,A,Worse
3,19,13.3,10,A,Worse
4,71,9.9,110,A,Improved
5,45,13.5,110,A,Worse


In [19]:
# Show last 5 rows from dataB
last(dataB, 5)

Row,Age,WCC,CRP,Treatment,Result
,Int64,Float64,Int64,String,String
1,40,9.7,80,B,Worse
2,74,11.0,10,B,Worse
3,33,10.7,70,B,Improved
4,39,12.0,70,B,Static
5,39,8.0,30,B,Worse


## Descriptive statistics using the dataframe object

The `describe()` function will attempt to provide descriptive statistics of the a data object.

In [20]:
describe(data)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,Age,51.74,18,54.5,80,0,Int64
2,WCC,11.634,7.3,12.0,16.1,0,Float64
3,CRP,39.7,0,30.0,110,0,Int64
4,Treatment,,A,,B,0,String
5,Result,,Improved,,Worse,0,String


We can count the number of the elements in the sample space of a categorical variable using the `combine()` function. 

**Note that previously used `by()` method has been deprecated**. Use `combine()` instead with the grouped dataframes.

In [21]:
# Define grouped data
grouped_df = groupby(data, :Treatment);

In [22]:
# Counting the number of patients in groups A and B
combine(grouped_df, nrow => :N)

Row,Treatment,N
,String,Int64
1,B,48
2,A,52


In [23]:
# The size argument will give the same output other than adding the number of variables i.e. 5 columns
# size() returns tuple containing row and column numbers
combine(size, grouped_df)

Row,Treatment,x1
,String,Tuple…
1,B,"(48, 5)"
2,A,"(52, 5)"


The usual descriptive statistics of a numerical variable can be calculated after separation by a categorical variable.

In [24]:
# Mean age of groups A and B patients
combine(grouped_df, :Age => mean)

Row,Treatment,Age_mean
,String,Float64
1,B,52.7083
2,A,50.8462


In [25]:
# Standard deviation of groups A and B patients
combine(grouped_df, :Age => std)

Row,Treatment,Age_std
,String,Float64
1,B,18.1377
2,A,18.4809


By using the `summarystats()` function we can get all the descriptive statistics.

In [26]:
combine(grouped_df, :Age => describe)

Summary Stats:
Length:         48
Missing Count:  0
Mean:           52.708333
Std. Deviation: 18.137722
Minimum:        21.000000
1st Quartile:   35.500000
Median:         56.000000
3rd Quartile:   68.500000
Maximum:        80.000000
Type:           Int64
Summary Stats:
Length:         52
Missing Count:  0
Mean:           50.846154
Std. Deviation: 18.480922
Minimum:        18.000000
1st Quartile:   35.750000
Median:         54.000000
3rd Quartile:   67.000000
Maximum:        80.000000
Type:           Int64


Row,Treatment,Age_describe
,String,Nothing
1,B,
2,A,


## Visualizing the data

The Plots package works well with the DataFrames package by allowing macro function from the latter.  In the code cell below, we look at the age distribution of the two treatment groups.

Note that `@df` macro (from StatsPlots) is used to pass the columns to the function.

In [27]:
@df data density(:Age, group = :Treatment, title = "Distribution of ages by treatment group",
    xlab = "Age", ylab = "Distribution",
    legend = :topright)

LoadError: LoadError: UndefVarError: `@df` not defined
in expression starting at c:\Users\chadi\OneDrive - Handelshögskolan i Stockholm\Corusera\JuliaCourseNotebooks\Julia_Week4\Julia 1.0 for statistics.ipynb:1

We can do the same for the results groups.

In [28]:
@df data density(:Age, group = :Result, title = "Distribution of ages by result group",
    xlab = "Age", ylab = "Distribution",
    legend = :topright)

LoadError: LoadError: UndefVarError: `@df` not defined
in expression starting at c:\Users\chadi\OneDrive - Handelshögskolan i Stockholm\Corusera\JuliaCourseNotebooks\Julia_Week4\Julia 1.0 for statistics.ipynb:1

We can even discriminate between all of the groups.

In [29]:
@df data density(:Age, group = (:Treatment, :Result), title = "Distribution of ages by treatment and result groups",
    xlab = "Age", ylab = "Distribution",
    legend = :topright)

LoadError: LoadError: UndefVarError: `@df` not defined
in expression starting at c:\Users\chadi\OneDrive - Handelshögskolan i Stockholm\Corusera\JuliaCourseNotebooks\Julia_Week4\Julia 1.0 for statistics.ipynb:1

Let's create a box-and-whisker plot of the white cell count per treatment group and then per result group.

In [30]:
@df data boxplot(:Treatment, :WCC, lab = "WCC", title = "White cell count by treatment group",
    xlab = "Groups", ylab = "WCC")

LoadError: LoadError: UndefVarError: `@df` not defined
in expression starting at c:\Users\chadi\OneDrive - Handelshögskolan i Stockholm\Corusera\JuliaCourseNotebooks\Julia_Week4\Julia 1.0 for statistics.ipynb:1

In [31]:
@df data boxplot(:Result, :WCC, lab = "WCC", title = "White cell count by result group",
    xlab = "Groups", ylab = "WCC")

LoadError: LoadError: UndefVarError: `@df` not defined
in expression starting at c:\Users\chadi\OneDrive - Handelshögskolan i Stockholm\Corusera\JuliaCourseNotebooks\Julia_Week4\Julia 1.0 for statistics.ipynb:1

Finally, we will check on the correlation between the numerical variables using a correlation plot and a corner plot.

In [32]:
@df data corrplot([:Age :WCC :CRP], grid = false)  # No comma's between arguments in list

LoadError: LoadError: UndefVarError: `@df` not defined
in expression starting at c:\Users\chadi\OneDrive - Handelshögskolan i Stockholm\Corusera\JuliaCourseNotebooks\Julia_Week4\Julia 1.0 for statistics.ipynb:1

In [33]:
@df data cornerplot([:Age :WCC :CRP], grid = false, compact = true)

LoadError: LoadError: UndefVarError: `@df` not defined
in expression starting at c:\Users\chadi\OneDrive - Handelshögskolan i Stockholm\Corusera\JuliaCourseNotebooks\Julia_Week4\Julia 1.0 for statistics.ipynb:1

## Inferential statistics

We will begin by using Student's _t_ test to compare the mean of a numerical variable between two groups. 

In [34]:
# Difference in age between patients in groups A and B
HypothesisTests.EqualVarianceTTest(dataA[:, :Age], dataB[:, :Age])

UndefVarError: UndefVarError: `HypothesisTests` not defined

In [35]:
# Only the p value for the difference in white cell count between patients in groups A and B
pvalue(EqualVarianceTTest(dataA[:, :WCC], dataB[:, :WCC]))

UndefVarError: UndefVarError: `EqualVarianceTTest` not defined

In [36]:
# Difference in c-reactive protein level between patients in groups A and B for unequal variances
UnequalVarianceTTest(dataA[:, :CRP], dataB[:, :CRP])

UndefVarError: UndefVarError: `UnequalVarianceTTest` not defined

We can create a variety of linear models using the `GLM.fit()` function.

In [37]:
# Simple model predicting CRP
fit(LinearModel, @formula(CRP ~ 1), data)

LoadError: LoadError: UndefVarError: `@formula` not defined
in expression starting at c:\Users\chadi\OneDrive - Handelshögskolan i Stockholm\Corusera\JuliaCourseNotebooks\Julia_Week4\Julia 1.0 for statistics.ipynb:2

In [38]:
# Adding Age as a predictor variable
fit(LinearModel, @formula(CRP ~ Age), data)

LoadError: LoadError: UndefVarError: `@formula` not defined
in expression starting at c:\Users\chadi\OneDrive - Handelshögskolan i Stockholm\Corusera\JuliaCourseNotebooks\Julia_Week4\Julia 1.0 for statistics.ipynb:2

In [39]:
# Adding Age and WCC as predictor variables
fit(LinearModel, @formula(CRP ~ Age + WCC), data)

LoadError: LoadError: UndefVarError: `@formula` not defined
in expression starting at c:\Users\chadi\OneDrive - Handelshögskolan i Stockholm\Corusera\JuliaCourseNotebooks\Julia_Week4\Julia 1.0 for statistics.ipynb:2

We can conduct a $\chi^2$ test for independence using the `HypothesisTests.ChisqTest()` function.  First we need to look at the counts.  Below we calculate the number of unique values for the Result variable sample space for patients in groups A and B.

Again note, that we use `combine()` instead of deprecated `by()` method.

In [40]:
combine(groupby(dataA, :Result), nrow => :N)

Row,Result,N
,String,Int64
1,Static,26
2,Worse,16
3,Improved,10


In [41]:
combine(groupby(dataB, :Result), nrow => :N)

Row,Result,N
,String,Int64
1,Improved,14
2,Worse,20
3,Static,14


In [42]:
# Enter the data in similar order here
observed = reshape([22, 17, 18, 18, 11, 14], (2, 3))
observed

2×3 Matrix{Int64}:
 22  18  11
 17  18  14

In [43]:
ChisqTest(observed)

UndefVarError: UndefVarError: `ChisqTest` not defined

## Exporting a CSV file

Finally we can export our dataframe object as a spreadsheet file.

In [44]:
CSV.write("ProjectData_1_point_0.csv", data);

-----